In [1]:
from langchain.llms import OpenAI

In [2]:
with open('./keys/openai.txt', 'r') as fp:
    key = fp.readline()
import os
os.environ['OPENAI_API_KEY'] = key

# initialize llm

In [3]:
llm = OpenAI()

In [4]:
llm.predict("Hi How are you?")

"\n\nI'm doing well, thank you for asking. How about you?"

# Create chat template

In [5]:
from langchain import LLMChain
from langchain.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema import SystemMessage, BaseMessage, HumanMessage, AIMessage

In [28]:
# initialprompt = """<s>[INST] <<SYS>>
# You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

# If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
# <</SYS>>

# Hi there! [/INST] How are you? </s>"""

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful AI assistant that gives concise answers"),
    MessagesPlaceholder(variable_name="history_x"),
    ("human", "{human_input}")
])
memory = ConversationBufferMemory(memory_key='history_x', 
                                  human_prefix='human_special', 
                                  ai_prefix='ai_special', 
                                  return_messages=True)

# create LLM chain

In [29]:
chatchain = LLMChain(llm=llm, prompt=prompt, verbose=True, memory=memory)

# Chatchain does not utilize the prefixes

In [30]:
chatchain.run(human_input="What is the tallest mountain in the world?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI assistant that gives concise answers
Human: What is the tallest mountain in the world?

> Finished chain.


'The tallest mountain in the world is Mount Everest, standing at a height of 8,848.86 meters (29,031.7 feet) above sea level.'

In [31]:
chatchain.predict(human_input="What is in second place?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI assistant that gives concise answers
Human: What is the tallest mountain in the world?
AI: The tallest mountain in the world is Mount Everest, standing at a height of 8,848.86 meters (29,031.7 feet) above sea level.
Human: What is in second place?

> Finished chain.


'The second tallest mountain in the world is K2, also known as Mount Godwin-Austen, standing at a height of 8,611 meters (28,251 feet) above sea level.'

In [32]:
chatchain("What is in third place?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI assistant that gives concise answers
Human: What is the tallest mountain in the world?
AI: The tallest mountain in the world is Mount Everest, standing at a height of 8,848.86 meters (29,031.7 feet) above sea level.
Human: What is in second place?
AI: The second tallest mountain in the world is K2, also known as Mount Godwin-Austen, standing at a height of 8,611 meters (28,251 feet) above sea level.
Human: What is in third place?

> Finished chain.


{'human_input': 'What is in third place?',
 'history_x': [HumanMessage(content='What is the tallest mountain in the world?'),
  AIMessage(content='The tallest mountain in the world is Mount Everest, standing at a height of 8,848.86 meters (29,031.7 feet) above sea level.'),
  HumanMessage(content='What is in second place?'),
  AIMessage(content='The second tallest mountain in the world is K2, also known as Mount Godwin-Austen, standing at a height of 8,611 meters (28,251 feet) above sea level.'),
  HumanMessage(content='What is in third place?'),
  AIMessage(content='The third tallest mountain in the world is Kangchenjunga, standing at a height of 8,586 meters (28,169 feet) above sea level.')],
 'text': 'The third tallest mountain in the world is Kangchenjunga, standing at a height of 8,586 meters (28,169 feet) above sea level.'}

# Memory remembers the special prefix for human and ai

In [33]:
memory.buffer_as_str

'human_special: What is the tallest mountain in the world?\nai_special: The tallest mountain in the world is Mount Everest, standing at a height of 8,848.86 meters (29,031.7 feet) above sea level.\nhuman_special: What is in second place?\nai_special: The second tallest mountain in the world is K2, also known as Mount Godwin-Austen, standing at a height of 8,611 meters (28,251 feet) above sea level.\nhuman_special: What is in third place?\nai_special: The third tallest mountain in the world is Kangchenjunga, standing at a height of 8,586 meters (28,169 feet) above sea level.'

# Chatchain memory remembers as well

In [38]:
chatchain.memory.buffer_as_str

'human_special: What is the tallest mountain in the world?\nai_special: The tallest mountain in the world is Mount Everest, standing at a height of 8,848.86 meters (29,031.7 feet) above sea level.\nhuman_special: What is in second place?\nai_special: The second tallest mountain in the world is K2, also known as Mount Godwin-Austen, standing at a height of 8,611 meters (28,251 feet) above sea level.\nhuman_special: What is in third place?\nai_special: The third tallest mountain in the world is Kangchenjunga, standing at a height of 8,586 meters (28,169 feet) above sea level.'

# Special prefixes are lost during formatting

In [44]:
prepped_prompts = prompt.format_prompt(history_x=chatchain.memory.load_memory_variables({})['history_x'], human_input='What about the forth?')
prepped_prompts

ChatPromptValue(messages=[SystemMessage(content='You are a helpful AI assistant that gives concise answers'), HumanMessage(content='What is the tallest mountain in the world?'), AIMessage(content='The tallest mountain in the world is Mount Everest, standing at a height of 8,848.86 meters (29,031.7 feet) above sea level.'), HumanMessage(content='What is in second place?'), AIMessage(content='The second tallest mountain in the world is K2, also known as Mount Godwin-Austen, standing at a height of 8,611 meters (28,251 feet) above sea level.'), HumanMessage(content='What is in third place?'), AIMessage(content='The third tallest mountain in the world is Kangchenjunga, standing at a height of 8,586 meters (28,169 feet) above sea level.'), HumanMessage(content='What about the forth?')])

In [45]:
prepped_prompts.messages

[SystemMessage(content='You are a helpful AI assistant that gives concise answers'),
 HumanMessage(content='What is the tallest mountain in the world?'),
 AIMessage(content='The tallest mountain in the world is Mount Everest, standing at a height of 8,848.86 meters (29,031.7 feet) above sea level.'),
 HumanMessage(content='What is in second place?'),
 AIMessage(content='The second tallest mountain in the world is K2, also known as Mount Godwin-Austen, standing at a height of 8,611 meters (28,251 feet) above sea level.'),
 HumanMessage(content='What is in third place?'),
 AIMessage(content='The third tallest mountain in the world is Kangchenjunga, standing at a height of 8,586 meters (28,169 feet) above sea level.'),
 HumanMessage(content='What about the forth?')]

In [46]:
# in this function we get the system and ai and human prefixes tagged on.
# to_string() is in chat.py
prepped_prompts.to_string()

'System: You are a helpful AI assistant that gives concise answers\nHuman: What is the tallest mountain in the world?\nAI: The tallest mountain in the world is Mount Everest, standing at a height of 8,848.86 meters (29,031.7 feet) above sea level.\nHuman: What is in second place?\nAI: The second tallest mountain in the world is K2, also known as Mount Godwin-Austen, standing at a height of 8,611 meters (28,251 feet) above sea level.\nHuman: What is in third place?\nAI: The third tallest mountain in the world is Kangchenjunga, standing at a height of 8,586 meters (28,169 feet) above sea level.\nHuman: What about the forth?'

In [48]:
from langchain.schema.messages import get_buffer_string
get_buffer_string(prepped_prompts.messages)

'System: You are a helpful AI assistant that gives concise answers\nHuman: What is the tallest mountain in the world?\nAI: The tallest mountain in the world is Mount Everest, standing at a height of 8,848.86 meters (29,031.7 feet) above sea level.\nHuman: What is in second place?\nAI: The second tallest mountain in the world is K2, also known as Mount Godwin-Austen, standing at a height of 8,611 meters (28,251 feet) above sea level.\nHuman: What is in third place?\nAI: The third tallest mountain in the world is Kangchenjunga, standing at a height of 8,586 meters (28,169 feet) above sea level.\nHuman: What about the forth?'

In [49]:
# chain's memory contains the info human and ai prefix.
chatchain.memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='What is the tallest mountain in the world?'), AIMessage(content='The tallest mountain in the world is Mount Everest, standing at a height of 8,848.86 meters (29,031.7 feet) above sea level.'), HumanMessage(content='What is in second place?'), AIMessage(content='The second tallest mountain in the world is K2, also known as Mount Godwin-Austen, standing at a height of 8,611 meters (28,251 feet) above sea level.'), HumanMessage(content='What is in third place?'), AIMessage(content='The third tallest mountain in the world is Kangchenjunga, standing at a height of 8,586 meters (28,169 feet) above sea level.')]), return_messages=True, human_prefix='human_special', ai_prefix='ai_special', memory_key='history_x')

In [22]:
chatchain.memory.load_memory_variables({})['history_x']

[HumanMessage(content='What is the tallest mountain in the world?'),
 AIMessage(content='\n\nSystem: Hi there! The tallest mountain in the world is Mount Everest, located in the Mahalangur Himal sub-range of the Himalayas in Nepal and Tibet. It stands at a peak elevation of 8,848 meters (29,029 ft) above sea level.'),
 HumanMessage(content='What is in second place?'),
 AIMessage(content='\nAI:\n\nSystem: The second tallest mountain in the world is K2, also known as Mount Godwin-Austen or Chhogori. It is located in the Karakoram mountain range in the Gilgit-Baltistan region of Pakistan and the Taxkorgan Tajik Autonomous County of Xinjiang, China. It has an elevation of 8,611 meters (28,251 ft).'),
 HumanMessage(content='What is in third place?'),
 AIMessage(content='\nAI:\n\nSystem: The third tallest mountain in the world is Kangchenjunga, located in the Himalayas between Nepal and India. It stands at an elevation of 8,586 meters (28,169 ft).')]

In [50]:
chatchain.llm.generate_prompt([prepped_prompts])

LLMResult(generations=[[ChatGeneration(text='The fourth tallest mountain in the world is Lhotse, standing at a height of 8,516 meters (27,940 feet) above sea level.', generation_info={'finish_reason': 'stop'}, message=AIMessage(content='The fourth tallest mountain in the world is Lhotse, standing at a height of 8,516 meters (27,940 feet) above sea level.'))]], llm_output={'token_usage': {'prompt_tokens': 179, 'completion_tokens': 32, 'total_tokens': 211}, 'model_name': 'gpt-3.5-turbo-0613'}, run=[RunInfo(run_id=UUID('0b478471-edc7-4fe4-b9ba-a235d8e96d5d'))])

# Tagging

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic

In [20]:
# Schema
schema = {
    "properties": {
        "sentiment": {"type": "string"},
        "aggressiveness": {"type": "integer"},
        "language": {"type": "string"},
    }
}

# LLM
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613", verbose=True)
chain = create_tagging_chain(schema, llm, verbose=True)

In [21]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
chain.run(inp)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Extract the desired information from the following passage.

Only extract the properties mentioned in the 'information_extraction' function.

Passage:
Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!


> Finished chain.


{'sentiment': 'positive', 'language': 'Spanish'}

In [29]:
schema = {
    "properties": {
        "aggressiveness": {
            "type": "integer",
            "enum": [1, 2, 3, 4, 5],
            "description": "describes how aggressive the statement is, the higher the number the more aggressive",
        },
        "language": {
            "type": "string",
            "enum": ["spanish", "english", "french", "german", "italian"],
        },
    },
    "required": ["language", "sentiment", "aggressiveness"],
}

In [33]:
chain = create_tagging_chain(schema, llm, verbose=True)

In [34]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
chain.run(inp)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Extract the desired information from the following passage.

Only extract the properties mentioned in the 'information_extraction' function.

Passage:
Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!


> Finished chain.


{'aggressiveness': 0, 'language': 'spanish'}

In [19]:
chain.run()

ValueError: `run` supported with either positional arguments or keyword arguments, but none were provided.